<h1>Capstone Data Science Final Project</h1>
<p>Welcome to Capstone Data Science Project. Here, I would present a brief summary of what I did in my final project for the capstone project. Toronto is the capital city of the Canadian province of Ontario having a recorded population of around 3 million. It is the most populous city in Canada and the fourth most populous city in North America. The fact behind it being so populus is that benefits it has compared to so many cities across North American Sub Continent, but also the opportunities provided by this city. Keeping in  mind the idea of building a new business (convinience store, cloth shop, restaurent, etc.) in this city keeping the location of other shopping malls in this city as the factor, i kept this project very basic. In real life this could be much complicated project involving lot of other factors, but here i have focused only on my data science skills.

First let's import the packages.</p>

In [1]:
import numpy as np 
import pandas as pd
import requests
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
!pip install beautifulsoup4
from bs4 import BeautifulSoup 
!pip install geopy
from geopy.geocoders import Nominatim
!pip install folium
import folium
import io

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

<h1>Data Retrieval</h1>
<p>The data about the names of the neighbourhoods of Toronto City is available here in this Wikipedia page along with their postal codes (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). We'll do data scraping from this link and now we put it in a dataframe.</p>

In [2]:
data_set = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data_set, "html.parser")
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"5b699626-93af-4ffb-947b-327b886fc099","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1013111980,"wgRevisionId":1013111980,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia semi-protected p

<h1>"Cleaning" parsed dataset</h1>

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

print(table_contents)

[{'Postal Code': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'Postal Code': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'Postal Code': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'Postal Code': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'Postal Code': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'Postal Code': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'Postal Code': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'Postal Code': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'Postal Code': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'Postal Code': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'Postal Code': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'Posta

<h1>Transform cleaned and prepared data to frame</h1>

In [4]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df = pd.DataFrame(table_contents)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<p>Here we have a list of Borough(administrative unit) and their surrounding neighbourhood.</p>

<h1>Size of dataframe</h1>

In [5]:
df.shape

(103, 3)

<h1>Adding latitudes and longitudes to frame</h1>

In [6]:
url = "http://cocl.us/Geospatial_data"
latlong_data = requests.get(url).content
lat_long = pd.read_csv(io.StringIO(latlong_data.decode("utf-8")))

df = pd.merge(df, lat_long)
df_toronto = df[["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]]
df_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<p>lets look at value counts:</p>

In [7]:
df['Borough'].value_counts()

North York                                                      24
Downtown Toronto                                                17
Scarborough                                                     17
Etobicoke                                                       11
Central Toronto                                                  9
West Toronto                                                     6
York                                                             5
East York                                                        4
East Toronto                                                     4
Queen's Park                                                     1
EtobicokeNorthwest                                               1
MississaugaCanada Post Gateway Processing Centre                 1
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                         

<p>The above data frame created we will use for the project.</p>

In [8]:
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<p>Now we will use geolocator to estimate location of Toronto city:</p>

In [9]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<p>Visualizing map of Toronto</p>

In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, df_toronto in zip(df_toronto['Latitude'],
                                         df_toronto['Longitude'], 
                                         df_toronto['Borough'], 
                                         df_toronto['Neighborhood']):
    label = '{}, {}'.format(df_toronto, borough)  
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<p>In this map the location of city along with neighborhoods have been marked. It is clearly visible from map.</p>

<h1>Lets take data of one borough, North York as it has max value counts</h1>

In [12]:
df_data = df[["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]]
downtown_data = df_data[df_data['Borough'] == 'North York'].reset_index(drop=True)
downtown_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,Don Mills South,43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


In [13]:
# coordinate of North York
address = 'North York'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


<p>Negative coordinate means location is in Western Hemisphere.</p>

<h1>Connecting with Foursquare API</h1>

In [14]:
CLIENT_ID = 'QFFTTVZUXKUBH4DRMXXU24ODDVIA222PT5QRTWL1LVQUWFDM' # your Foursquare ID
CLIENT_SECRET = 'PC1LVBJTPNHHK0RTA0DCBT2EXTIRMSY0KHUC5W50F3W4KJPB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QFFTTVZUXKUBH4DRMXXU24ODDVIA222PT5QRTWL1LVQUWFDM
CLIENT_SECRET:PC1LVBJTPNHHK0RTA0DCBT2EXTIRMSY0KHUC5W50F3W4KJPB


In [17]:
downtown_data.loc[0, 'Neighborhood']

'Parkwoods'

In [18]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = downtown_data.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


<p>Here we will take top 50 venues for radius of 300 m(for business location)</p>

In [19]:
# requesting data from foursquare
LIMIT = 50
radius = 300
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    VERSION, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=QFFTTVZUXKUBH4DRMXXU24ODDVIA222PT5QRTWL1LVQUWFDM&client_secret=PC1LVBJTPNHHK0RTA0DCBT2EXTIRMSY0KHUC5W50F3W4KJPB&ll=43.7532586,-79.3296565&v=20180605&radius=300&limit=50'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6065ba71c07d6c2a4015fbfd'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.755958602700005,
    'lng': -79.32592554390956},
   'sw': {'lat': 43.7505585973, 'lng': -79.33338745609043}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
# Clean the json and structure it into a dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.33214


In [23]:
# returned venues 
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data["Latitude"],
                                   longitudes=downtown_data['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Glencairn
Don Mills South
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [26]:
print(downtown_venues.shape)
downtown_venues

(226, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
...,...,...,...,...,...,...,...
221,Willowdale West,43.782736,-79.442259,Tov-Li,43.784214,-79.446098,Pizza Place
222,Willowdale West,43.782736,-79.442259,Shoppers Drug Mart,43.784847,-79.446028,Pharmacy
223,Willowdale West,43.782736,-79.442259,Tim Hortons,43.780940,-79.444231,Coffee Shop
224,Willowdale West,43.782736,-79.442259,Price Chopper,43.783237,-79.446339,Grocery Store


In [27]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Don Mills North,4,4,4,4,4,4
Don Mills South,21,21,21,21,21,21
Downsview Central,3,3,3,3,3,3
Downsview East,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,3,3,3,3,3,3


In [28]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 94 uniques categories.


<h1>Analyzing all neighbourhoods</h1>

<p>In Order to analyse all neighborhoods, we will be using one-hot encoding first. Since computers are very primitive and only understand binary values of '1' and '0', it is difficult to be apply machine learning to categorical data.The benefit with one-hot encoding is that we convert this categorical data into numerical data. Then all this data is converted into binary vectors.For this purpose, values are first mapped into integer values. After that each integer value is then represented as a binary vector.</p>

In [29]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
downtown_onehot.shape

(226, 95)

<p>Now we will group them by 'Neighbourhoods'</p>

In [31]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.086957,...,0.000000,0.043478,0.00,0.043478,0.000000,0.00,0.00,0.00,0.000000,0.00
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.250000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.0,0.00,0.000000,...,0.000000,0.000000,0.00,0.041667,0.041667,0.00,0.00,0.00,0.000000,0.00
3,Don Mills North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00
4,Don Mills South,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619,0.0,0.00,0.000000,...,0.047619,0.047619,0.00,0.047619,0.000000,0.00,0.00,0.00,0.000000,0.00
5,Downsview Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00
6,Downsview East,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00
7,Downsview Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.00,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00
8,Downsview West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.333333,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00
9,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.020000,0.000000,0.000000,0.020000,0.0,0.02,0.040000,...,0.020000,0.000000,0.02,0.000000,0.000000,0.02,0.02,0.02,0.000000,0.02


<h1>Data for each neighborhood along with the top 5 most common venues</h1>

In [33]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop  0.09
1           Bank  0.09
2       Pharmacy  0.04
3  Shopping Mall  0.04
4           Park  0.04


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.08
2      Sandwich Place  0.08
3           Juice Bar  0.08
4             Butcher  0.04


----Don Mills North----
                  venue  freq
0                   Gym  0.25
1  Caribbean Restaurant  0.25
2                  Café  0.25
3   Japanese Restaurant  0.25
4                Lounge  0.00


----Don Mills South----
            venue  freq
0     Coffee Shop  0.10
1             Gym  0.10
2      Restaurant  0.10
3      Beer Store  0.05
4  Discount Store  0.05


----Downsview Ce

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Bridal Shop,Park,Pizza Place,Mobile Phone Shop,Deli / Bodega,Ice Cream Shop,Middle Eastern Restaurant,Restaurant
1,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,"Bedford Park, Lawrence Manor East",Juice Bar,Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Café,Butcher,Indian Restaurant,Pet Store,Pharmacy
3,Don Mills North,Japanese Restaurant,Caribbean Restaurant,Café,Gym,Women's Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,Don Mills South,Gym,Coffee Shop,Restaurant,Sandwich Place,Clothing Store,Italian Restaurant,Bike Shop,Beer Store,Dim Sum Restaurant,Discount Store
5,Downsview Central,Food Truck,Home Service,Baseball Field,Women's Store,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
6,Downsview East,Park,Airport,Business Service,Dim Sum Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
7,Downsview Northwest,Discount Store,Grocery Store,Gym / Fitness Center,Athletics & Sports,Liquor Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
8,Downsview West,Park,Bank,Shopping Mall,Dim Sum Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
9,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Bank,Mobile Phone Shop,Restaurant,Juice Bar,Liquor Store,Movie Theater,Burger Joint


<h1>Clustering the Neighborhoods Using k-means Clustering</h1>
<p>'k-means clustering' is a machine learning technique used for clustering data. We have selected the number of centroids for clustering, i.e. the number of clusters we want to grouo the data into, or the value for 'k'.The data points are grouped into the cluster of the closest centroid(Expectation). Then the new centroid/mean of the cluster is calculated(Maximization). The Expectation Step and Maximization step are in an iteration and continue until none of the data points are left that need to be grouped, or the centroid for each cluster does not change. Let's use this technique for clustering the neighborhoods for this project.
We will keep k = 5</p>

In [36]:
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 2, 4, 0, 0, 0], dtype=int32)

In [37]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge downtown_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Food & Drink Shop,Park,Women's Store,Department Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Women's Store,Department Store,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Boutique,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Comfort Food Restaurant
3,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Japanese Restaurant,Caribbean Restaurant,Café,Gym,Women's Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Pizza Place,Park,Bakery,Japanese Restaurant,Department Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping


<h1>Final Part:Examining Clusters</h1>

In [40]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Women's Store,Department Store,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
3,North York,0.0,Japanese Restaurant,Caribbean Restaurant,Café,Gym,Women's Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,North York,0.0,Pizza Place,Park,Bakery,Japanese Restaurant,Department Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
5,North York,0.0,Gym,Coffee Shop,Restaurant,Sandwich Place,Clothing Store,Italian Restaurant,Bike Shop,Beer Store,Dim Sum Restaurant,Discount Store
6,North York,0.0,Golf Course,Mediterranean Restaurant,Pool,Fast Food Restaurant,Dog Run,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
7,North York,0.0,Coffee Shop,Bank,Bridal Shop,Park,Pizza Place,Mobile Phone Shop,Deli / Bodega,Ice Cream Shop,Middle Eastern Restaurant,Restaurant
8,North York,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Bank,Mobile Phone Shop,Restaurant,Juice Bar,Liquor Store,Movie Theater,Burger Joint
9,North York,0.0,Miscellaneous Shop,Vietnamese Restaurant,Furniture / Home Store,Caribbean Restaurant,Coffee Shop,Metro Station,Massage Studio,Bar,Department Store,Comfort Food Restaurant
10,North York,0.0,Chinese Restaurant,Japanese Restaurant,Café,Bank,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
13,North York,0.0,Park,Bank,Shopping Mall,Dim Sum Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


In [41]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Lawrence Manor, Lawrence Heights",1.0,Clothing Store,Accessories Store,Boutique,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Comfort Food Restaurant


In [42]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downsview Central,2.0,Food Truck,Home Service,Baseball Field,Women's Store,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
19,"Humberlea, Emery",2.0,Construction & Landscaping,Baseball Field,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega


In [43]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,"Willowdale, Newtonbrook",3.0,Piano Bar,Women's Store,Caribbean Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [44]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,4.0,Food & Drink Shop,Park,Women's Store,Department Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
11,Downsview East,4.0,Park,Airport,Business Service,Dim Sum Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
22,York Mills West,4.0,Convenience Store,Park,Women's Store,Dim Sum Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop


<h1>Conclusion</h1>
<p>As it was already mentioned above that the algorithm was set for radius of 300m, an ideal distance kept in mind to have sufficient space to start a business venture. Keeping in mind above parameter and analysing different value of cluster labels, we have analysed our data. The clusters show different type business ventures present in that area.The cluster label 0 is having very large no of different types of businesses operating. Hence opening up another buisness will be really tough and non profitable. The cluster label 2 and 4 are having a couple of business operations. Setting up an opportunity other than they are currently having could be a good idea as well as a profitable business depending of day by day needs of locals. The cluster labels 1 and 3 are barely having any business ventures hence all kind of ideas would be effective and the most profitable one would be the one which locals are demanding.</p>

<h1>Final Remarks</h1>
<p>Here i would like to conclude my brief Data Science Project and its Analysis. I hope it will provide necessary insights to whomever reading this notebook.</p>
<p>Thank you for being here and reading this project!</p>